# Heartbeat Multiclass classification

In this notebook, we will focus on healthcare. This data set is made available by MIT. It contains data about 9,026 heartbeat measurements. Each row represents a single measurement (captured on a timeline). There are a total of 80 data points (columns). This is a multiclass classification task: predict whether the measurement represents a normal heartbeat or other anomalies. 

## Description of Variables

You will use the **hearbeat_cleaned.csv** data set for this assignment. Each row represents a single measurement. Columns labeled as T1 from T80 are the time steps on the timeline (there are 80 time steps, each time step has only one measurement). 

The last column is the target variable. It shows the label (category) of the measurement as follows:<br>
0 = Normal<br>
1 = Supraventricular premature beat<br>
2 = Premature ventricular contraction<br>
3 = Fusion of ventricular and normal beat<br>
4 = Unclassifiable beat

## Goal

Use the data set **hearbeat_cleaned.csv** to predict the column called **Target**. The input variables are columns labeled as **T1 to T80**. 

# Note:

The data is cleaned up. There are no unqueal length sequences. And, there is no zero padding. So, there is no need to use any `Masking` layer.

# Read and Prepare the Data 

In [20]:
# Common imports
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd

In [21]:
#We will predict the "Target" value in the data set:

heartbeat = pd.read_csv("heartbeat_cleaned.csv")
heartbeat.head()

,T1,T2,T3,T4,T5,T6,T7,T8,T9,T10,...,T72,T73,T74,T75,T76,T77,T78,T79,T80,Target
0,0.987,0.892,0.461,0.1130,0.1490,0.1900,0.1650,0.1620,0.1470,0.1380,...,0.1970,0.1970,0.1960,0.2030,0.201,0.1990,0.2010,0.205,0.2080,0
1,1.000,0.918,0.621,0.1330,0.1050,0.1250,0.1170,0.0898,0.0703,0.0781,...,0.1950,0.1910,0.1520,0.1720,0.207,0.2110,0.2070,0.207,0.1720,0
2,1.000,0.751,0.143,0.1040,0.0961,0.0519,0.0442,0.0416,0.0364,0.0857,...,0.2260,0.2420,0.2440,0.2860,0.468,0.8160,0.9770,0.452,0.0519,0
3,1.000,0.740,0.235,0.0464,0.0722,0.0567,0.0103,0.0155,0.0284,0.0155,...,0.0851,0.0747,0.0515,0.0593,0.067,0.0361,0.1210,0.451,0.8690,0
4,1.000,0.833,0.309,0.0191,0.1010,0.1200,0.1040,0.0874,0.0765,0.0765,...,0.2050,0.4210,0.8030,0.9510,0.467,0.0000,0.0519,0.082,0.0628,0


In [22]:
heartbeat.shape

(7960, 81)

In [23]:
y = heartbeat['Target']
x = heartbeat.drop('Target', axis=1)

In [24]:
#Train test split
from sklearn.model_selection import train_test_split

train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.3)

Data Transformation


In [25]:
#Target variables need to be an array with integer type
train_y = np.array(train_y)
test_y = np.array(test_y)

train_y = train_y.astype(np.int32)
test_y = test_y.astype(np.int32)


In [26]:
test_y.dtype

dtype('int32')

In [27]:
#Convert input variables to a 2-D array with float data type
train_x= np.array(train_x)
test_x= np.array(test_x)

train_x = train_x.astype(np.float32)
test_x = test_x.astype(np.float32)

In [28]:
train_x

array([[0.961 , 0.783 , 0.549 , ..., 0.132 , 0.18  , 0.442 ],
       [1.    , 0.938 , 0.751 , ..., 0.143 , 0.14  , 0.15  ],
       [0.889 , 0.489 , 0.278 , ..., 0.15  , 0.05  , 0.0778],
       ...,
       [0.958 , 0.942 , 0.34  , ..., 0.175 , 0.229 , 0.331 ],
       [1.    , 0.897 , 0.323 , ..., 0.222 , 0.239 , 0.249 ],
       [0.939 , 0.843 , 0.254 , ..., 0.307 , 0.304 , 0.282 ]],
      dtype=float32)

In [29]:
#Keras expects a different input format:
#Data needs to have 3 dimensions

train_x = np.reshape(train_x, (train_x.shape[0], train_x.shape[1], 1))
test_x = np.reshape(test_x, (test_x.shape[0], test_x.shape[1], 1))



In [30]:
train_x.shape, train_y.shape

((5572, 80, 1), (5572,))

In [31]:
train_x

array([[[0.961 ],
        [0.783 ],
        [0.549 ],
        ...,
        [0.132 ],
        [0.18  ],
        [0.442 ]],

       [[1.    ],
        [0.938 ],
        [0.751 ],
        ...,
        [0.143 ],
        [0.14  ],
        [0.15  ]],

       [[0.889 ],
        [0.489 ],
        [0.278 ],
        ...,
        [0.15  ],
        [0.05  ],
        [0.0778]],

       ...,

       [[0.958 ],
        [0.942 ],
        [0.34  ],
        ...,
        [0.175 ],
        [0.229 ],
        [0.331 ]],

       [[1.    ],
        [0.897 ],
        [0.323 ],
        ...,
        [0.222 ],
        [0.239 ],
        [0.249 ]],

       [[0.939 ],
        [0.843 ],
        [0.254 ],
        ...,
        [0.307 ],
        [0.304 ],
        [0.282 ]]], dtype=float32)

In [62]:
train_y

array([0, 2, 1, ..., 0, 0, 1])

In [63]:
test_y

array([0, 1, 2, ..., 4, 0, 4])

# Find the baseline 

In [34]:
heartbeat['Target'].value_counts()/len(heartbeat)

0    0.582035
4    0.198995
2    0.155402
1    0.055905
3    0.007663
Name: Target, dtype: float64

# Build a cross-sectional shallow model using Keras (with only one hidden layer) 

In [35]:
model = keras.models.Sequential([
    
    keras.layers.Flatten(input_shape=[80, 1]),
    keras.layers.Dense(40, activation='relu'),
    keras.layers.Dense(5, activation='softmax')
    
])

In [36]:
from tensorflow.keras.callbacks import EarlyStopping


earlystop = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='auto')

callback = [earlystop]

In [37]:
np.random.seed(45)
tf.random.set_seed(45)
optimizer = tf.keras.optimizers.Nadam(learning_rate=0.01)

# If multiclass, use "sparse_categorical_crossentropy" as the loss function
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=['accuracy'])


history = model.fit(train_x, train_y, epochs=100,
                    validation_data=(test_x, test_y),callbacks=callback)

Epoch 1/100
175/175 [==============================] - 1s 3ms/step - loss: 0.7170 - accuracy: 0.7629 - val_loss: 0.5235 - val_accuracy: 0.8275
Epoch 2/100
175/175 [==============================] - 0s 1ms/step - loss: 0.4744 - accuracy: 0.8444 - val_loss: 0.6891 - val_accuracy: 0.6993
Epoch 3/100
175/175 [==============================] - 0s 1ms/step - loss: 0.4203 - accuracy: 0.8711 - val_loss: 0.5329 - val_accuracy: 0.7956
Epoch 4/100
175/175 [==============================] - 0s 1ms/step - loss: 0.3676 - accuracy: 0.8898 - val_loss: 0.4029 - val_accuracy: 0.8652
Epoch 5/100
175/175 [==============================] - 0s 1ms/step - loss: 0.3365 - accuracy: 0.9031 - val_loss: 0.3082 - val_accuracy: 0.9112
Epoch 6/100
175/175 [==============================] - 0s 1ms/step - loss: 0.3175 - accuracy: 0.9067 - val_loss: 0.5399 - val_accuracy: 0.8664
Epoch 7/100
175/175 [==============================] - 0s 1ms/step - loss: 0.3011 - accuracy: 0.9112 - val_loss: 0.3721 - val_accuracy: 0.8807

In [38]:
# evaluate the model

scores = model.evaluate(test_x, test_y, verbose=0)

scores

# In results, first is loss, second is accuracy

[0.26640573143959045, 0.9292294979095459]

In [39]:
# extract the accuracy from model.evaluate

print("%s: %.2f" % (model.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))


loss: 0.27
accuracy: 92.92%


# Build a cross-sectional deep model using Keras (with two or more hidden layers) 

In [40]:
#PIPE Architecture

model = keras.models.Sequential()

model.add(keras.layers.Input(shape=80))
model.add(keras.layers.Dense(80, activation='relu'))
model.add(keras.layers.Dense(80, activation='relu'))
model.add(keras.layers.Dense(80, activation='relu'))
model.add(keras.layers.Dense(5, activation='softmax'))

#final layer: there has to be 5 nodes with softmax (because we have 5 categories)

In [41]:
np.random.seed(45)
tf.random.set_seed(45)
# Compile model

#Optimizer:
adam = keras.optimizers.Adam(learning_rate=0.01)

model.compile(loss='sparse_categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

In [42]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 80)                6480      
                                                                 
 dense_3 (Dense)             (None, 80)                6480      
                                                                 
 dense_4 (Dense)             (None, 80)                6480      
                                                                 
 dense_5 (Dense)             (None, 5)                 405       
                                                                 
Total params: 19,845
Trainable params: 19,845
Non-trainable params: 0
_________________________________________________________________


In [43]:
# Fit the model

history = model.fit(train_x, train_y, 
                    validation_data=(test_x, test_y), 
                    epochs=100, batch_size=1000,callbacks=callback)

Epoch 1/100
6/6 [==============================] - 0s 25ms/step - loss: 1.2297 - accuracy: 0.4928 - val_loss: 0.9177 - val_accuracy: 0.6696
Epoch 2/100
6/6 [==============================] - 0s 9ms/step - loss: 0.8180 - accuracy: 0.7452 - val_loss: 0.6827 - val_accuracy: 0.8074
Epoch 3/100
6/6 [==============================] - 0s 7ms/step - loss: 0.6524 - accuracy: 0.7933 - val_loss: 0.6201 - val_accuracy: 0.7961
Epoch 4/100
6/6 [==============================] - 0s 7ms/step - loss: 0.5716 - accuracy: 0.8123 - val_loss: 0.5090 - val_accuracy: 0.8484
Epoch 5/100
6/6 [==============================] - 0s 7ms/step - loss: 0.5038 - accuracy: 0.8369 - val_loss: 0.4576 - val_accuracy: 0.8576
Epoch 6/100
6/6 [==============================] - 0s 6ms/step - loss: 0.4483 - accuracy: 0.8580 - val_loss: 0.4181 - val_accuracy: 0.8647
Epoch 7/100
6/6 [==============================] - 0s 7ms/step - loss: 0.4149 - accuracy: 0.8724 - val_loss: 0.3855 - val_accuracy: 0.8840
Epoch 8/100
6/6 [=========

In [44]:
# evaluate the model

scores = model.evaluate(test_x, test_y, verbose=0)

scores

# In results, first is loss, second is accuracy

[0.23832276463508606, 0.9367671608924866]

In [45]:
# extract the accuracy from model.evaluate

print("%s: %.2f" % (model.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))


loss: 0.24
accuracy: 93.68%


# Build a sequential shallow LSTM Model (with only one LSTM layer) 

In [46]:
n_steps = 80
n_inputs = 1

model = keras.models.Sequential([
    
    keras.layers.LSTM(5, activation='softmax' , input_shape=[n_steps, n_inputs])
])

In [47]:
np.random.seed(45)
tf.random.set_seed(45)
optimizer = keras.optimizers.Nadam(learning_rate=0.01)

model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=['accuracy'])

history = model.fit(train_x, train_y, epochs=100,
                   validation_data = (test_x, test_y), callbacks=callback)

Epoch 1/100
175/175 [==============================] - 4s 16ms/step - loss: 1.1619 - accuracy: 0.5720 - val_loss: 1.0708 - val_accuracy: 0.5930
Epoch 2/100
175/175 [==============================] - 3s 15ms/step - loss: 1.0567 - accuracy: 0.5817 - val_loss: 1.0695 - val_accuracy: 0.5733
Epoch 3/100
175/175 [==============================] - 3s 15ms/step - loss: 1.0507 - accuracy: 0.5772 - val_loss: 1.0566 - val_accuracy: 0.6089
Epoch 4/100
175/175 [==============================] - 3s 15ms/step - loss: 1.0441 - accuracy: 0.5813 - val_loss: 1.0698 - val_accuracy: 0.6005
Epoch 5/100
175/175 [==============================] - 3s 17ms/step - loss: 1.0412 - accuracy: 0.5881 - val_loss: 1.0499 - val_accuracy: 0.5787
Epoch 6/100
175/175 [==============================] - 3s 16ms/step - loss: 1.0377 - accuracy: 0.5800 - val_loss: 1.0589 - val_accuracy: 0.6009
Epoch 7/100
175/175 [==============================] - 4s 21ms/step - loss: 1.0349 - accuracy: 0.5808 - val_loss: 1.0517 - val_accuracy:

In [48]:
# evaluate the model

scores = model.evaluate(test_x, test_y, verbose=0)

scores

# In results, first is loss, second is accuracy

[0.7014719247817993, 0.7763819098472595]

In [49]:
# extract the accuracy from model.evaluate

print("%s: %.2f" % (model.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))


loss: 0.70
accuracy: 77.64%


# Build a sequential deep LSTM Model (with only two LSTM layers)

In [70]:
n_steps = 80
n_inputs = 1

model = keras.models.Sequential([
    keras.layers.LSTM(9, return_sequences=True, input_shape=[n_steps, n_inputs]),
    keras.layers.LSTM(9, return_sequences=False),
    keras.layers.Dense(5, activation='softmax')
])

In [71]:
np.random.seed(45)
tf.random.set_seed(45)
optimizer = keras.optimizers.Nadam(learning_rate=0.01)

model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=['accuracy'])

history = model.fit(train_x, train_y, epochs=100,
                   validation_data = (test_x, test_y), callbacks=callback)

Epoch 1/100
175/175 [==============================] - 8s 31ms/step - loss: 1.1408 - accuracy: 0.5806 - val_loss: 1.1270 - val_accuracy: 0.5854
Epoch 2/100
175/175 [==============================] - 5s 29ms/step - loss: 1.1017 - accuracy: 0.5879 - val_loss: 1.0943 - val_accuracy: 0.5854
Epoch 3/100
175/175 [==============================] - 5s 30ms/step - loss: 1.0654 - accuracy: 0.5885 - val_loss: 1.0466 - val_accuracy: 0.6227
Epoch 4/100
175/175 [==============================] - 5s 30ms/step - loss: 1.0477 - accuracy: 0.6168 - val_loss: 1.0478 - val_accuracy: 0.6298
Epoch 5/100
175/175 [==============================] - 5s 30ms/step - loss: 1.0391 - accuracy: 0.6023 - val_loss: 1.0957 - val_accuracy: 0.6160
Epoch 6/100
175/175 [==============================] - 5s 30ms/step - loss: 1.0257 - accuracy: 0.6131 - val_loss: 0.9610 - val_accuracy: 0.6621
Epoch 7/100
175/175 [==============================] - 5s 30ms/step - loss: 0.8167 - accuracy: 0.7062 - val_loss: 0.7062 - val_accuracy:

In [72]:
# evaluate the model

scores = model.evaluate(test_x, test_y, verbose=0)

scores

# In results, first is loss, second is accuracy

[0.30231717228889465, 0.9162479043006897]

In [73]:
# extract the accuracy from model.evaluate

print("%s: %.2f" % (model.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))


loss: 0.30
accuracy: 91.62%


# Build a sequential shallow GRU Model (with only one GRU layer) 

In [74]:
n_steps = 80
n_inputs = 1

model = keras.models.Sequential([
    keras.layers.GRU(2, input_shape=[n_steps, n_inputs]),
    keras.layers.Dense(5, activation='softmax')
])

In [75]:
np.random.seed(45)
tf.random.set_seed(45)

optimizer = keras.optimizers.Nadam(learning_rate=0.01)

model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=['accuracy'])

history = model.fit(train_x, train_y, epochs=100,
                   validation_data = (test_x, test_y), callbacks=callback)

Epoch 1/100
175/175 [==============================] - 6s 23ms/step - loss: 1.1526 - accuracy: 0.5696 - val_loss: 1.0540 - val_accuracy: 0.6055
Epoch 2/100
175/175 [==============================] - 4s 21ms/step - loss: 1.0402 - accuracy: 0.6037 - val_loss: 1.0532 - val_accuracy: 0.5733
Epoch 3/100
175/175 [==============================] - 4s 21ms/step - loss: 1.0264 - accuracy: 0.5992 - val_loss: 1.0293 - val_accuracy: 0.6244
Epoch 4/100
175/175 [==============================] - 4s 22ms/step - loss: 1.0167 - accuracy: 0.5948 - val_loss: 1.0248 - val_accuracy: 0.6047
Epoch 5/100
175/175 [==============================] - 4s 21ms/step - loss: 1.0120 - accuracy: 0.5973 - val_loss: 1.0135 - val_accuracy: 0.6076
Epoch 6/100
175/175 [==============================] - 4s 21ms/step - loss: 1.0065 - accuracy: 0.6023 - val_loss: 1.0094 - val_accuracy: 0.5817
Epoch 7/100
175/175 [==============================] - 4s 21ms/step - loss: 1.0013 - accuracy: 0.5973 - val_loss: 1.0033 - val_accuracy:

In [76]:
# evaluate the model

scores = model.evaluate(test_x, test_y, verbose=0)

scores

# In results, first is loss, second is accuracy

[0.8059611320495605, 0.694304883480072]

In [77]:
# extract the accuracy from model.evaluate

print("%s: %.2f" % (model.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))


loss: 0.81
accuracy: 69.43%


# Build a sequential deep GRU Model (with only two GRU layers) 

In [78]:
n_steps = 80
n_inputs = 1

model = keras.models.Sequential([
    keras.layers.GRU(7, return_sequences=True, input_shape=[n_steps, n_inputs]),
    keras.layers.GRU(7, return_sequences=False),
    keras.layers.Dense(5, activation='softmax')
])

In [79]:
np.random.seed(45)
tf.random.set_seed(45)

optimizer = keras.optimizers.Nadam(learning_rate=0.01)

model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=['accuracy'])

history = model.fit(train_x, train_y, epochs=100,
                   validation_data = (test_x, test_y), callbacks=callback)

Epoch 1/100
175/175 [==============================] - 9s 39ms/step - loss: 1.1332 - accuracy: 0.5800 - val_loss: 1.0554 - val_accuracy: 0.6055
Epoch 2/100
175/175 [==============================] - 8s 45ms/step - loss: 1.0208 - accuracy: 0.5944 - val_loss: 1.0580 - val_accuracy: 0.5854
Epoch 3/100
175/175 [==============================] - 6s 36ms/step - loss: 0.9802 - accuracy: 0.6184 - val_loss: 0.8627 - val_accuracy: 0.6859
Epoch 4/100
175/175 [==============================] - 7s 43ms/step - loss: 0.7284 - accuracy: 0.7421 - val_loss: 0.6693 - val_accuracy: 0.7885
Epoch 5/100
175/175 [==============================] - 8s 44ms/step - loss: 0.6034 - accuracy: 0.8098 - val_loss: 0.5413 - val_accuracy: 0.8405
Epoch 6/100
175/175 [==============================] - 7s 39ms/step - loss: 0.5533 - accuracy: 0.8293 - val_loss: 0.6117 - val_accuracy: 0.8040
Epoch 7/100
175/175 [==============================] - 7s 41ms/step - loss: 0.5953 - accuracy: 0.8047 - val_loss: 0.7267 - val_accuracy:

In [80]:
# evaluate the model

scores = model.evaluate(test_x, test_y, verbose=0)

scores

# In results, first is loss, second is accuracy

[0.32454797625541687, 0.9074539542198181]

In [81]:
# extract the accuracy from model.evaluate

print("%s: %.2f" % (model.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))


loss: 0.32
accuracy: 90.75%


# Discussion

## test values of each model you built 

                 Model	                          Validation Accuracy
    cross-sectional shallow model (1 hidden layer)	       92.92%

    cross-sectional deep model (3 hidden layers)	         93.68%

    sequential shallow LSTM model (1 LSTM layer)	         77.64%

    sequential deep LSTM model (2 LSTM layers)	           91.62%

    sequential shallow GRU model (1 GRU layer)	           69.43%

    sequential deep GRU model (2 GRU layers)	             90.75%


## Which model performs the best and why? 

The cross-sectional deep model with 3 hidden layers (PIPE Architecture) performs the best. It has the highest validation accuracy (93.68%) among all the models.


## How does it compare to baseline? 

The majority class (0) is about 58.20% of the data. Anything above this should be a good model for us. The best model is 35.48% above the baseline.
